In [7]:
import import_ipynb
import copy
from builtins import range

import corner as Co
import edge as Ed
import facelet as Fc
import facecube as FC
import numpy as np

In [36]:
# moves
cpU = [Co.UBR, Co.URF, Co.UFL, Co.ULB, Co.DFR, Co.DLF, Co.DBL, Co.DRB]
coU = np.array([0, 0, 0, 0, 0, 0, 0, 0])
epU = [Ed.UB, Ed.UR, Ed.UF, Ed.UL, Ed.DR, Ed.DF, Ed.DL, Ed.DB, Ed.FR, Ed.FL, Ed.BL, Ed.BR]
eoU = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

cpR = [Co.DFR, Co.UFL, Co.ULB, Co.URF, Co.DRB, Co.DLF, Co.DBL, Co.UBR]
coR = np.array([2, 0, 0, 1, 1, 0, 0, 2])
epR = [Ed.FR, Ed.UF, Ed.UL, Ed.UB, Ed.BR, Ed.DF, Ed.DL, Ed.DB, Ed.DR, Ed.FL, Ed.BL, Ed.UR]
eoR = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

cpF = [Co.UFL, Co.DLF, Co.ULB, Co.UBR, Co.URF, Co.DFR, Co.DBL, Co.DRB]
coF = np.array([1, 2, 0, 0, 2, 1, 0, 0])
epF = [Ed.UR, Ed.FL, Ed.UL, Ed.UB, Ed.DR, Ed.FR, Ed.DL, Ed.DB, Ed.UF, Ed.DF, Ed.BL, Ed.BR]
eoF = np.array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0])

cpD = [Co.URF, Co.UFL, Co.ULB, Co.UBR, Co.DLF, Co.DBL, Co.DRB, Co.DFR]
coD = np.array([0, 0, 0, 0, 0, 0, 0, 0])
epD = [Ed.UR, Ed.UF, Ed.UL, Ed.UB, Ed.DF, Ed.DL, Ed.DB, Ed.DR, Ed.FR, Ed.FL, Ed.BL, Ed.BR]
eoD = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

cpL = [Co.URF, Co.ULB, Co.DBL, Co.UBR, Co.DFR, Co.UFL, Co.DLF, Co.DRB]
coL = np.array([0, 1, 2, 0, 0, 2, 1, 0])
epL = [Ed.UR, Ed.UF, Ed.BL, Ed.UB, Ed.DR, Ed.DF, Ed.FL, Ed.DB, Ed.FR, Ed.UL, Ed.DL, Ed.BR]
eoL = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

cpB = [Co.URF, Co.UFL, Co.UBR, Co.DRB, Co.DFR, Co.DLF, Co.ULB, Co.DBL]
coB = np.array([0, 0, 1, 2, 0, 0, 2, 1])
epB = [Ed.UR, Ed.UF, Ed.UL, Ed.BR, Ed.DR, Ed.DF, Ed.DL, Ed.BL, Ed.FR, Ed.FL, Ed.UB, Ed.DB]
eoB = np.array([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1])

In [49]:
def corner_num_to_str_translator(s):
    a = []
    for i in range(len(s)):
        for c in Co.corners.keys():
            if s[i] == Co.corners[c]:
                a.append(c)
                break
    return np.array(a)    

def edge_num_to_str_translator(s):
    a = []
    for i in range(len(s)):
        for e in Ed.edges.keys():
            if s[i] == Ed.edges[e]:
                a.append(e)
                break
    return np.array(a)



class CubieCube(object):
    #cubie level
    def __init__(self, cp=None, co=None, ep=None, eo=None):
        # corner permutation
        self.cp = copy.copy(cp) if cp else [Co.URF, Co.UFL, Co.ULB, Co.UBR, Co.DFR, Co.DLF, Co.DBL, Co.DRB]
        self.cp_str = corner_num_to_str_translator(cp)
        
        # corner orientation
        if co.all() != None:
            self.co = co.copy()  
        else:
            np.array([0, 0, 0, 0, 0, 0, 0, 0])

        # edge permutation
        self.ep = copy.copy(ep) if ep else [Ed.UR, Ed.UF, Ed.UL, Ed.UB, Ed.DR, Ed.DF, Ed.DL, Ed.DB, Ed.FR, Ed.FL, Ed.BL, Ed.BR]
        self.ep_str = edge_num_to_str_translator(ep)
        
        # edge orientation
        if eo.all()!= None:
            self.eo = eo.copy()
        else:
            np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])        
    
    def __str__(self):
        return str(self.to_facelet())
    
    def to_2dstring(self):
        return self.to_facelet().to_2dstring()
    
    def to_facelet(self):
        s = ['0' for i in range(54)]
        # set the centers
        s[Fc.U5] = 'U'
        s[Fc.R5] = 'R'
        s[Fc.F5] = 'F'
        s[Fc.D5] = 'D'
        s[Fc.L5] = 'L'
        s[Fc.B5] = 'B'
        
        # assign the value by each cubie
        # corner
        for c in Co.corner_keys:
            Cubie_at_c = self.cp_str[Co.corners[c]]
            color_at_face = ''
            for i in range(3):
                face = Co.corner_facelets[c][i]
                index = i
                if self.co[Co.corners[c]] % 3 == 1:  # clockwise
                    index = (i - 1 + 3)%3
                if self.co[Co.corners[c]] % 3 == 2:  # counterclockwise
                    index = (i - 2 + 3)%3   
                num_of_color_at_face = Co.corner_facelets[Cubie_at_c][index]
                for i in Fc.facelets.keys():
                    if Fc.facelets[i] == num_of_color_at_face:
                        color_at_face = i
                s[face] = color_at_face        
        # edge
        for e in Ed.edge_keys:
            Cubie_at_e = self.ep_str[Ed.edges[e]]
            color_at_face = ''
            for i in range(2):
                face = Ed.edge_facelets[e][i]
                index = i
                if self.eo[Ed.edges[e]] % 2 == 1:  # flipped
                    index = (i - 1 + 2) % 2
                num_of_color_at_face = Ed.edge_facelets[Cubie_at_e][index]
                for i in Fc.facelets.keys():
                    if Fc.facelets[i] == num_of_color_at_face:
                        color_at_face = i
                s[face] = color_at_face        

        seq = "".join(s)   
        Q = FC.FaceCube(seq)
        return Q         

In [43]:
corner_num_to_str_translator(cpU)

array(['UBR', 'URF', 'UFL', 'ULB', 'DFR', 'DLF', 'DBL', 'DRB'],
      dtype='<U3')

In [50]:
a = CubieCube(cpU, coU, epU,eoU)
print(a.to_2dstring())

   UUU
   UUU
   UUU
FFFRRRBBBLLL
LLLFFFRRRBBB
LLLFFFRRRBBB
   DDD
   DDD
   DDD

